# Importamos Librerías

In [1]:
from tkinter import *
from tkinter import messagebox, filedialog
from tkinter import ttk
import tkinter as tk
import os, sys
import hashlib
import random
from sys import getsizeof
import linecache
import base64
import time
from tkinter.messagebox import showinfo

from cryptography.hazmat.backends import default_backend
from cryptography import x509
from cryptography.x509 import load_pem_x509_certificate
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from datetime import datetime
import datetime as dt
import random

In [2]:
from uuid import uuid4

def generateUUID():
    return str(uuid4())

# Paths

In [3]:
key_path = os.path.join("keys")
txt_path = os.path.join("txt")

# Variables Iniciales

In [4]:
q=int(0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff)
a=int(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
b=int(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
Px=int(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296)
Py=int(0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
n=int(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551)

# Funciones Modulares

In [5]:
def sum_(Px,Py,Qx,Qy):
    lambd=((Qy-Py)*modinverse_(Qx-Px,q))%q
    Rx=(lambd**2-Px-Qx)%q
    Ry=(-Py+lambd*(Px-Rx))%q
    return Rx,Ry

def duplicate_(Px,Py):
    lambd=((3*(Px**2)+a)*modinverse_(2*Py,q))%q
    Qx=(lambd**2-2*Px)%q
    Qy=(-Py+lambd*(Px-Qx))%q
    return Qx,Qy

def binaryscalarproduct_(Px,Py,k):
    kbin=bin(k)[2:]
    #print('kbin:', kbin)
    m=len(kbin)
    #print(m)
    Qx=Px
    Qy=Py
    #for j in range(m-2,-1,-1):
    for j in range(1,m,1):
        #print('Secuencia: ',j,' Bit: ',kbin[j])
        Qx,Qy=duplicate_(Qx,Qy)
        #print('duplicado:',Qx,Qy)
        if kbin[j]=='1':
            Qx,Qy=sum_(Qx,Qy,Px,Py)
            #print('sumado:',Qx,Qy)
    return Qx,Qy

def modinverse_(w,c):#w es el número que se desea invertir; c es el módulo
    x = pow(w,c-2,c)
    return x

def verify_():
    cond=(4*a**3+27*b**2)%q
    if cond!=0:
        exist=True
    else:
        exist=False
    return exist

# Estilo

In [6]:
raiz = Tk()

raiz.title("Reto: Firmas Digitales")
raiz.resizable(0, 0)
raiz.geometry("600x300")


#cuadrillo del tec con titulo xd
myFrame = Frame()
myFrame.pack(side="top")
myFrame.config(bg="#F2F2F2")

text = Label(
    text="Instituto Tecnológico de Estudios Superiores de Monterrey\n\n Teletón\n\n "
)
text.place(x=125, y=20)


text = Label(
    text="Elige una opción: \n"
)
text.place(x=200, y=80)

combo = ttk.Combobox(raiz)
combo.place(x=200, y=100)
combo.config(justify= "center")
combo["values"] = ("Generar Certificado y Clave", "Generar Firma", "Verificar Firma")

# Generación de Llaves

In [7]:
def generatePrivateKey_(n):
    d = random.randint(1,n-1)
    return d

def generatePublicKey_(Px,Py,d):
    Dx,Dy=binaryscalarproduct_(Px,Py,d)
    return Dx,Dy

def generar_llaves():
    savingName = os.path.join(key_path, "private_key.pem")
    file_out = open(savingName, "w")
    d = generatePrivateKey_(n)
    file_out.write(str(d))
    file_out.close()
    
    savingName = os.path.join(key_path, "public_key.pem")
    file_out = open(savingName, "w")
    Dx, Dy = generatePublicKey_(Px,Py,d)
    file_out.write('Clave Publica Dx: '+str(Dx))
    file_out.write('\n')
    file_out.write('Clave Publica Dy: '+str(Dy))
    file_out.close()

# Seleccionar Opción

# Abrir Archivos

In [8]:
def abrirArchivo_a_Usar():
    raiz.archivo = filedialog.askopenfilename(
        initialdir=txt_path,
        title="Selecciona un archivo txt para firmar o para verificar",
        filetypes=(("txt files", ".txt"), ("all files", ".*")),
    )

def abrirArchivo_a_Usar_f():
    raiz.llave = filedialog.askopenfilename(
        initialdir = key_path,
        title = "Selecciona una llave",
        filetypes = (("pem files", ".pem"), ("all files", ".*")),
    )

def abrirArchivo_a_Usar_c():
    raiz.certificado = filedialog.askopenfilename(
        initialdir=key_path,
        title="Selecciona el certificado",
        filetypes=(("DER files", ".der"), ("all files", ".*")),
    )

# Estilo del Menú

In [9]:
def seleccionar_funcion():
    combo_sel = combo.get()

    if combo_sel == "Generar Firma":
        firma()

    elif combo_sel == "Verificar Firma":
        verificar()

    elif combo_sel == "Generar Certificado y Clave":
        generar_llaves()
        certificado()

    else:
        messagebox.showinfo("Error ", "You must select an option")

In [10]:
start = Button(raiz, text="Aceptar", command=seleccionar_funcion, justify="center", anchor="center")
start.place(x=200, y=180, anchor="center")

### Firma

In [11]:
def firma():
    newWindow = Toplevel(raiz)
    newWindow.title("Generación de Firma")
    newWindow.geometry("500x300")
        
    def ventanaFirmas():
        nuevaVentana = Toplevel(raiz)
        nuevaVentana.title("Firma Digital: ")
        nuevaVentana.geometry("300x150")
        labelV = Label(
            nuevaVentana, 
            text='Firma Digital:'
            )
        labelV.grid(row=0, column=1, padx=10)
        
        botonF = Button(
            nuevaVentana, 
            text='Descargar Documento')
        botonF.grid(row=2, columnspan=3, pady=10)

    def uploadFiles():
        root = tk.Toplevel()
        root.geometry('300x120')
        root.title('Cargando')
        
        pb1 = ttk.Progressbar(
            root, 
            orient= "horizontal", 
            length=300, 
            mode='determinate',
            )
        pb1.grid(row=5, columnspan=3, pady=20)

        for i in range(5):
            root.update_idletasks()
            Label(root, text=f"Progreso: {pb1['value']}%").grid(column=0, row=6, columnspan=2)
            pb1['value'] += 20
            time.sleep(1)
            Label(root, text=f"Progreso: {pb1['value']}%").grid(column=0, row=6, columnspan=2)
        pb1.destroy()
        Label(root, text='Archivo Subido', foreground='green').grid(row=5, columnspan=3, padx=10, pady=10)

        with open(file_save, encoding="utf-8") as f:
            message = f.readlines()
            message = "".join(message)
        message_to_sign = message
        generate_signature(message_to_sign)

    x = Label(newWindow,
        text = "Por favor, llena la siguiente información")
    x.grid(row=0, column=1, padx=10)
    
    archivo = Label(
        newWindow, 
        text='Archivo por Firmar: '
        )
    archivo.grid(row=1, column=0, padx=10)

    boton_arch = Button(
        newWindow, 
        text="Seleccionar Archivo",
        command= abrirArchivo_a_Usar
        ) 
    boton_arch.grid(row=1, column=1)

    cp = Label(
        newWindow, 
        text='Clave privada:'
        )
    cp.grid(row=2, column=0, padx=10)

    cp_boton = Button(
        newWindow, 
        text="Seleccionar Archivo",
        command= abrirArchivo_a_Usar_f
        ) 
    cp_boton.grid(row=2, column=1)
    
    upld = Button(
        newWindow, 
        text='Enviar archivos', 
        command=uploadFiles
        )
    upld.grid(row=4, columnspan=3, pady=10)    
    file_save = os.path.join(txt_path, "Mensaje.txt")



### Ventana de Verificación

In [12]:
# ventana de verificar firma 
def verificar():
    newWindow = Toplevel(raiz)
    newWindow.title("Verificación de Firma")
    newWindow.geometry("500x300")

    def uploadFiles():
        root = tk.Toplevel()
        root.geometry('300x120')
        root.title('Cargando')
        
        pb1 = ttk.Progressbar(
            root, 
            orient= "horizontal", 
            length=300, 
            mode='determinate',
            )
        pb1.grid(row=5, columnspan=3, pady=20)

        for i in range(5):
            root.update_idletasks()
            Label(root, text=f"Progreso: {pb1['value']}%").grid(column=0, row=6, columnspan=2)
            pb1['value'] += 20
            time.sleep(1)
            Label(root, text=f"Progreso: {pb1['value']}%").grid(column=0, row=6, columnspan=2)
        pb1.destroy()
        Label(root, text='Archivo Subido', foreground='green').grid(row=5, columnspan=3, padx=10, pady=10)

        file_save = os.path.join(txt_path, "Mensaje.txt")
        with open(file_save, encoding="utf-8") as f:
            message = f.readlines()
            message = "".join(message)

        botonVer = Button(
            root,
            text='Verificar Firma',
            command=verify_signature(message))
        botonVer.grid(row=7, columnspan=3, pady=10)

    def ventanaVer():
        nuevaVentana = Toplevel(raiz)
        nuevaVentana.title("Verificación ")
        nuevaVentana.geometry("300x150")
        labelV = Label(
            nuevaVentana, 
            text='Resultado de la verificación:',
            font = 'Times 15 bold')
        labelV.grid(row=0, column=1, padx=10)
        
        labelR = Label(
            nuevaVentana, 
            text='TRUE',
            font = 'Times 15 bold')
        labelR.grid(row=2, column=1, padx=10)

        
    x = Label(newWindow,
          text = "Por favor, llena la siguiente información")
    x.grid(row=0, column=1, padx=10)
    
    archivo = Label(
        newWindow, 
        text='Archivo por verificar: '
        )
    archivo.grid(row=1, column=0, padx=10)

    boton_arch = Button(
        newWindow, 
        text="Seleccionar Archivo",
        command= abrirArchivo_a_Usar
        ) 
    boton_arch.grid(row=1, column=1)

    cp = Label(
        newWindow, 
        text='Certificado:'
        )
    cp.grid(row=2, column=0, padx=10)

    cp_boton = Button(
        newWindow, 
        text="Seleccionar Archivo",
        command= abrirArchivo_a_Usar_c
        ) 
    cp_boton.grid(row=2, column=1)    
    
    archivo = Label(
        newWindow, 
        text='Clave Pública: '
        )
    archivo.grid(row=3, column=0, padx=10)

    boton_arch = Button(
        newWindow, 
        text="Seleccionar Archivo",
        command= abrirArchivo_a_Usar_f
        ) 
    boton_arch.grid(row=3, column=1)
    
    upld = Button(
        newWindow, 
        text='Enviar archivos', 
        command=uploadFiles
        )
    upld.grid(row=6, columnspan=3, pady=10)

### Certificado

In [13]:
def certificado():
    newWindow = Toplevel(raiz)
    newWindow.title("Generación de Certificado")
 
    newWindow.geometry("400x200")
    newWindow.resizable(False,False)

    def uploadFiles(window):
        pb1 = ttk.Progressbar(
            window, 
            orient=HORIZONTAL, 
            length=300, 
            mode='determinate'
            )
        pb1.grid(row=5, columnspan=3, pady=20)
        for i in range(5):
            window.update_idletasks()
            pb1['value'] += 20
            time.sleep(1)
        pb1.destroy()
    
    x = Label(newWindow,
          text = "Por favor, llena la siguiente información")
    x.grid(row=0, column=1, padx=10)
    
    nl = Label(newWindow,
          text = "Nombre: ")
    nl.grid(row=1, column=0, padx=10)
    global nomb
    nomb = tk.Entry(newWindow)
    nomb.grid(row=1, column=1, padx=10)
    
    el = Label(newWindow,
          text = "Estado: ")
    el.grid(row=3, column=0, padx=10)
    global estb
    estb = tk.Entry(newWindow)
    estb.grid(row=3, column=1, padx=10)
    
    cl = Label(newWindow,
          text = "Ciudad: ")
    cl.grid(row=4, column=0, padx=10)
    global ciudadb
    ciudadb = tk.Entry(newWindow)
    ciudadb.grid(row=4, column=1, padx=10)
    
    ol = Label(newWindow,
          text = "Organización: ")
    ol.grid(row=5, column=0, padx=10)
    global orgb
    orgb = tk.Entry(newWindow)
    orgb.grid(row=5, column=1, padx=10)

    cl = Label(newWindow,
          text = "Correo electrónico: ")
    cl.grid(row=6, column=0, padx=10)
    global corb
    corb = tk.Entry(newWindow)
    corb.grid(row=6, column=1, padx=10)
    
    def generate_certificate(_=None):
        global cor
        global ciudad
        global org
        global est
        global nom
        
        cor = str(corb.get())
        ciudad = ciudadb.get()
        org = orgb.get()
        ciudad = ciudadb.get()
        est = estb.get()
        nom = str(nomb.get())
        
        public_a = os.path.join(key_path,"public_key.pem") 
        pukey = open(public_a)
        Lines = pukey.readlines()
        for i in Lines:
            if (i[:18]) == 'Clave Publica Dx: ':
                Dx = int(i[18:])
            elif (i[:18]) == 'Clave Publica Dy: ':
                Dy = int(i[18:])

        curve = ec.SECP256R1()
        publickey = ec.EllipticCurvePublicNumbers(Dx, Dy, curve)
        objectpublickey = publickey.public_key()
    
        priv_a = os.path.join(key_path,"private_key.pem")
        prikey = open(priv_a).read()
        d = int(prikey)
        privatekey = ec.derive_private_key(d, curve)
        
       
        builder = x509.CertificateBuilder().subject_name(x509.Name([
            x509.NameAttribute(x509.oid.NameOID.COUNTRY_NAME, "MX"),
            x509.NameAttribute(x509.oid.NameOID.STATE_OR_PROVINCE_NAME, est),
            x509.NameAttribute(x509.oid.NameOID.LOCALITY_NAME,ciudad),
            x509.NameAttribute(x509.oid.NameOID.ORGANIZATION_NAME, org),
            x509.NameAttribute(x509.oid.NameOID.COMMON_NAME, nom),
            x509.NameAttribute(x509.oid.NameOID.EMAIL_ADDRESS, cor),
        ])).issuer_name(x509.Name([
            x509.NameAttribute(x509.oid.NameOID.COUNTRY_NAME, "MX"),
            x509.NameAttribute(x509.oid.NameOID.STATE_OR_PROVINCE_NAME, "NUEVO LEON"),
            x509.NameAttribute(x509.oid.NameOID.LOCALITY_NAME, "MONTERREY"),
            x509.NameAttribute(x509.oid.NameOID.ORGANIZATION_NAME, "TEC DE MONTERREY"),
            x509.NameAttribute(x509.oid.NameOID.COMMON_NAME, "EQUIPO 3"),
            x509.NameAttribute(x509.oid.NameOID.EMAIL_ADDRESS, "teleton@tec.org"),
        ])).not_valid_before(dt.datetime.today() - dt.timedelta(days=1))\
            .not_valid_after(dt.datetime.today() + dt.timedelta(days=365 * 2))\
            .serial_number(int(uuid4()))\
            .public_key(objectpublickey)\
            .add_extension(x509.BasicConstraints(ca=True, path_length=None), critical=True)

        certificate = builder.sign(
            private_key=privatekey,
            algorithm=hashes.SHA256(),
            backend=default_backend()
        )

        savingCert = os.path.join(key_path, "certificado.der")
        file_cer = open(savingCert, "wb")
        file_cer.write(certificate.public_bytes(encoding=serialization.Encoding.DER ))
        file_cer.close()


    botonF1 = Button(newWindow, 
        text='Generar', 
        command= generate_certificate
        )
    botonF1.grid(row=8, columnspan=3, pady=10)

# Hash

In [14]:
def hash_function(message):
    u=hashlib.sha256()
    u.update(message.encode())
    u=int(u.hexdigest(),16)

    return u

# Firmas

## Generar Firma

In [15]:
def generate_signature(message_to_sign):
    print("Generating Signature")

    priv_a = os.path.join(key_path, "private_key.pem")
    key = open(priv_a).read()
    d = int(key)
    
    r=0
    s=0
    while r==0 or s==0:
        k=random.randint(0,n-1) #para fines de esta implementación, k se da en los vectores de prueba.
        Qx,Qy=binaryscalarproduct_(Px,Py,k)
        r=Qx%n
        h = hash_function(message_to_sign)
        j=modinverse_(k,n)%n
        s=(j*(h+d*r))%n
        print(r,s)

    signs = os.path.join(txt_path, "MensajeFirmado.txt")
    signed_file = open(signs, "w", encoding="utf8")

    Mensaje = os.path.join(txt_path,"Mensaje.txt")
    with open(Mensaje, encoding="utf-8") as f:
        message = f.readlines()
        message = "".join(message)
    signed_file.write(message)
    signed_file.write("\n\n")
    signed_file.write('Firma Digital (r): '+str(r))
    signed_file.write("\n")
    signed_file.write('Firma Digital (s): '+str(s))
    signed_file.close()

    return r,s


## Verificar Firma

In [16]:
def verify_signature(message):
    
    #Lee la publica
    public_a = os.path.join(key_path,"public_key.pem")
    key = open(public_a)
    Lines = key.readlines()
    for i in Lines:
        if (i[:18]) == 'Clave Publica Dx: ':
            Dx = int(i[18:])
        elif (i[:18]) == 'Clave Publica Dy: ':
            Dy = int(i[18:])

    #Lee la firma digital
    firma_dig = os.path.join(txt_path,"MensajeFirmado.txt")
    key = open(firma_dig,'r')       
    
    Lines = key.readlines()
    for i in Lines:
        if (i[:19]) == 'Firma Digital (s): ':
            s = int(i[19:])
        elif (i[:19]) == 'Firma Digital (r): ':
            r = int(i[19:])

    
    h = hash_function(message)
    print(message)
    print('')
    print(h)
    if (r<=0 or r>=(n-1) or s<=0 or s>=(n-1)):
        messagebox.showinfo("Error", "Signature not valid")
    else:
        w=modinverse_(s,n)%n
        u1=(w*h)%n
        u2=(r*w)%n
        u1_times_Px,u1_times_Py=binaryscalarproduct_(Px,Py,u1)
        u2_times_Dx,u2_times_Dy=binaryscalarproduct_(Dx,Dy,u2)
        Yx,Yy=sum_(u1_times_Px,u1_times_Py,u2_times_Dx,u2_times_Dy)
        v=Yx%n
        print('r: ',r)
        print('s: ',s)
        print('Dx: ',Dx)
        print('Dy: ',Dy)
        
        print(v)
        if (r==v):
            print(h)
            messagebox.showinfo("Success", "Message verified correctly valid signature")
        else:
            messagebox.showinfo("Error", "Signature not valid")

# Regresar

In [ ]:
raiz.mainloop()

Generating Signature
10916469878860137974195023097466782893428554000969963207407394441254161486542 46262488992248136233398094340592753900391360998210985849861059524808932847456
sample

79232240492262066599341792208678897019497196617930543451364792016062498329023
r:  10916469878860137974195023097466782893428554000969963207407394441254161486542
s:  46262488992248136233398094340592753900391360998210985849861059524808932847456
Dx:  73278918427418512813408432221062846231698744794858903382785795461563591809060
Dy:  32974107215476604440143580410290783634504742442866236265190844330404844835759
10916469878860137974195023097466782893428554000969963207407394441254161486542
79232240492262066599341792208678897019497196617930543451364792016062498329023
